<a href="https://colab.research.google.com/github/maverick98/Coursera/blob/master/astar_romania.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
 heuristic_distance = {
                        'Arad': 366,
                        'Bucharest': 0,
                        'Craiova': 160,
                        'Drobeta': 242,
                        'Eforie': 161,
                        'Fagaras': 176,
                        'Giurgiu': 77,
                        'Hirsova': 151,
                        'Iasi': 226,
                        'Lugoj': 244,
                        'Mehadia': 241,
                        'Neamt': 234,
                        'Oradea': 380,
                        'Pitesti': 100,
                        'Rimnicu Vilcea': 193,
                        'Sibiu': 253,
                        'Timisoara': 329,
                        'Urziceni': 80,
                        'Vaslui': 199,
                        'Zerind': 374
                        }

In [8]:
class Node:
    def __init__(self,key,value,parent=None):
        self.key=key
        self.value=value
        self.parent=None
        self.idx=0

    def priority(self):
        return self.key+heuristic_distance[self.value]
    def set_parent(self,parent):
        self.parent=parent
    def __str__(self):
        return 'Value={}, Key={}'.format(self.value,self.key)
class PriorityQueue:
    def __init__(self):
        self.queue=[None]
        self.size=0
    def parent(self,i):
        return int(i/2)
    def left_child(self,i):
        return 2*i
    def right_child(self,i):
        return 2*i+1
    def exchange(self,i,j):
        self.queue[i],self.queue[j]=self.queue[j],self.queue[i]
        self.queue[i].idx,self.queue[j].idx=self.queue[j].idx,self.queue[i].idx
    def less(self,i,j):
        return self.queue[i].priority() < self.queue[j].priority()
    def more(self,i,j):
        return self.queue[i].priority() > self.queue[j].priority()
    def swim(self,k):
        if self.size ==1:
           return
        while(k > 1 and self.more(self.parent(k),k)):
             self.exchange(k,self.parent(k))
             k=self.parent(k)
    def sink(self,k):
        while self.left_child(k) <= self.size:
              left_child_idx= self.left_child(k)
              idx=left_child_idx
              right_child_idx= self.right_child(k)

              if right_child_idx <= self.size and self.more(left_child_idx,right_child_idx):
                 idx=right_child_idx
              if self.more(k,idx):
                 self.exchange(k,idx)
              k=idx

    def is_empty(self):
        return self.size==0
    def resize(self,new_size):
        if new_size >self.size:
           queue_resized=[None]*(new_size)
           for  i in range(self.size+1):
                queue_resized[i]=self.queue[i]
           self.queue=queue_resized

    def min(self):
        if self.is_empty() == False:
            return self.queue[1]
        return None
    def show(self):
         print('Priority Queue size is {}'.format(self.size))
         for i in range(self.size+1):
            if i >0:
                print(self.queue[i])

    def insert(self,node):
        if self.size == (len(self.queue)-1):
            self.resize(2*len(self.queue))

        self.size=self.size+1
        #print('Inserting node {} '.format(node))
        node.idx=self.size
        self.queue[self.size]=node


        self.swim(self.size)

    def extract_min(self):
        if self.is_empty() == False:
            min=self.min()
            self.exchange(1,self.size)
            self.size=self.size-1
            self.sink(1)
            if self.is_empty() == True:
               self.queue=[None]

            return min

        return None
    def contains_value(self,value):
        for i in range(1,self.size+1):
            if self.queue[i].value == value:
                return True
        return False




In [10]:

class Graph:

     def build_graph(self):
        all_nodes ={}
        self.graph={}
        for key_str in self.graph_str:
            neighbours=[]
            for (m,weight) in self.graph_str[key_str]:
                neighbour=None
                if m not in all_nodes.keys():
                    neighbour=Node(0,m)
                    all_nodes[m]=neighbour
                else:
                    neighbour=all_nodes[m]
                neighbours.append((neighbour,weight))
            self.graph[key_str]=neighbours

     def get_neighbors(self,v):
         if v in self.graph:
            return self.graph[v]
         else:
            return None


     def aStarSearch(self,start_node, stop_node):
         open_set=PriorityQueue()
         closed_set=set()
         open_set.insert(start_node)
         while open_set.is_empty() == False:
               #n=open_set.min()
               n=open_set.extract_min()
               if n == None:
                  print('Path does not exist!')
                  return None
               if n.value == stop_node.value:
                    path = []
                    stop_node_clone=n
                    while n.parent is not None:
                        path.append(n.value)
                        n = n.parent
                    path.append(start_node.value)
                    path.reverse()
                    print('Cost is {}'.format(stop_node_clone.key))
                    return path
               neighbours=graph.get_neighbors(n.value)
               #print('Min node is {}'.format(n.value))
               for (m,weight) in neighbours:
                    if m.value not in closed_set and open_set.contains_value(m.value) == False:
                       g_m=n.key+weight
                       m.key=g_m
                       m.parent=n
                       #print('Inserting node is {} '.format(m))
                       open_set.insert(m)

                    else:
                        if m.key > n.key+weight:
                           m.key=n.key+weight
                           m.parent=n
                           #print('Swimming up {} with cost {}'.format(m.idx,m.key))
                           #print('less cost {} for {}'.format(m.key,m))
                           open_set.swim(m.idx)
                           if m.value in closed_set:
                              closed_set.remove(m.value)
                              #print('Inserting node is {} due to less cost'.format(n))
                              open_set.insert(m)


               closed_set.add(n.value)
         return None
     def __init__(self):
        self.graph_str = {
                        'Arad': [('Zerind', 75), ('Sibiu', 140), ('Timisoara', 118)],
                        'Zerind': [('Arad', 75), ('Oradea', 71)],
                        'Sibiu': [('Arad', 140), ('Oradea', 151),('Fagaras', 99),('Rimnicu Vilcea', 80)],
                        'Timisoara': [('Arad', 118), ('Lugoj', 111)],
                        'Oradea': [('Zerind', 71), ('Sibiu', 151)],
                        'Fagaras': [('Bucharest', 211), ('Sibiu', 99)],
                        'Rimnicu Vilcea': [('Craiova', 146), ('Sibiu', 80), ('Pitesti', 97)],
                        'Lugoj': [('Timisoara', 111), ('Mehadia', 70)],
                        'Craiova': [('Rimnicu Vilcea', 146), ('Drobeta', 120), ('Pitesti', 138)],
                        'Pitesti': [('Rimnicu Vilcea', 97), ('Craiova', 138), ('Bucharest', 101)],
                        'Mehadia': [('Drobeta', 75), ('Lugoj', 70)],
                        'Drobeta': [('Mehadia', 75), ('Craiova', 120)],
                        'Bucharest': [('Pitesti', 101), ('Urziceni', 85),('Fagaras', 211),('Giurgiu', 90)],
                        'Urziceni': [('Bucharest', 85), ('Vaslui', 142), ('Hirsova', 98)],
                        'Giurgiu': [('Bucharest', 90)],
                        'Vaslui': [('Urziceni', 142), ('Iasi', 92)],
                        'Iasi': [('Vaslui', 92),('Neamt', 87)],
                        'Neamt': [('Iasi', 87)],
                        'Hirsova': [('Urziceni', 98), ('Eforie', 86)],
                        'Eforie': [('Hirsova', 86)]


                        }

        self.graph={}

graph=Graph()
graph.build_graph()
start_node=Node(0,'Arad')
stop_node=Node(0,'Bucharest')
path=graph.aStarSearch(start_node,stop_node)
print('Path found: {}'.format(path))


Cost is 418
Path found: ['Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest']
